In [1]:
import urllib
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier


In [39]:
for x in range(2013,2021):
    f = open(f"nov{x}-{x+1}.txt", "r")
    content = f.read()
    f.close()

    content = content.replace(" ","")

    g = open(f"nov{x}-{x+1}.txt", "w")
    g.write(content)
    g.close()

In [40]:
frames = []

for x in range(2013,2021):
    frames.append(pd.read_csv(f"nov{x}-{x+1}.txt", sep = ","))

data = pd.concat(frames)

In [41]:
data.sample(10)

,Aeg,Temperatuur,Niiskus,&Otilde;hur&otilde;hk,Tuulekiirus,Tuulesuund,Sademed,UVindeks,Valgustatus,Kiirgusvoog,Radioaktiivsus
36002,2016-03-1823:15:00,-6.862533,68.568590,1007.165633,2.997421,294.391644,0.000,NaN,NaN,0.000000,NaN
2787,2015-11-2416:15:00,0.296330,90.288050,1013.935233,2.549722,201.963629,0.000,NaN,NaN,0.000000,NaN
71755,2018-07-2204:30:00,17.298835,96.882466,1007.979500,1.938233,294.947038,0.000,NaN,NaN,2.040086,NaN
36123,2014-03-2011:00:00,-0.997132,14.757291,1000.675429,2.002985,303.383481,0.000,NaN,NaN,NaN,0.104565
11682,2020-12-2513:30:00,-0.527283,98.961514,1000.654400,2.716490,7.470259,0.050,NaN,NaN,23.396217,NaN
77035,2019-08-0414:15:00,13.766824,63.252625,1009.446133,3.527925,288.056271,0.000,NaN,NaN,309.403915,NaN
5238,2018-12-0304:30:00,-7.831291,94.875449,1007.182267,4.289932,114.577360,0.000,NaN,NaN,5.057494,NaN
1477,2016-11-2003:05:00,2.933741,97.747194,1007.454800,4.250906,112.004958,0.112,NaN,NaN,0.000000,NaN
72604,2016-07-2402:25:00,17.682881,93.854321,1019.441233,2.337995,6.372329,0.000,NaN,NaN,2.855848,NaN
19014,2019-01-1502:10:00,-4.706036,91.084722,980.580433,3.610237,271.840485,0.000,NaN,NaN,4.072501,NaN


In [42]:
data.shape

(811086, 11)

In [2]:
#f = open("kiirgusvoog_5a.csv", "r")
#content = f.read()
#f.close()

#content = content.replace(" ","")

#g = open("kiirgusvoog_5a_2.csv", "w")
#g.write(content)
#g.close()

#data = pd.read_csv("kiirgusvoog_5a_2.csv")

In [43]:
data = data.rename(columns = {'&Otilde;hur&otilde;hk':'Õhurõhk'})

In [44]:
def break_up_time(data):
    data["Aasta"]= data.apply(lambda row: row["Aeg"][0:4], axis=1)
    data["Kuu"] = data.apply(lambda row: row["Aeg"][5:7], axis=1)
    data["Päev"] = data.apply(lambda row: row["Aeg"][8:10], axis=1)
    data["Tund"] = data.apply(lambda row: row["Aeg"][10:12], axis=1)
    data["Minut"] = data.apply(lambda row: row["Aeg"][13:15], axis=1)
    data.Aasta = data.Aasta.astype('int')
    data.Kuu = data.Kuu.astype('int')
    data.Päev = data.Päev.astype('int')
    data.Tund = data.Tund.astype('int')
    data.Minut = data.Minut.astype('int')

In [45]:
break_up_time(data)
data.head()

,Aeg,Temperatuur,Niiskus,Õhurõhk,Tuulekiirus,Tuulesuund,Sademed,UVindeks,Valgustatus,Kiirgusvoog,Radioaktiivsus,Aasta,Kuu,Päev,Tund,Minut
0,2013-11-1500:00:00,2.253725,80.280623,1012.974940,1.396552,219.132229,0.0,NaN,NaN,NaN,0.097777,2013,11,15,0,0
1,2013-11-1500:05:00,2.269765,80.633583,1013.042850,1.246345,220.840096,0.0,NaN,NaN,NaN,0.134524,2013,11,15,0,5
2,2013-11-1500:10:00,2.259058,80.844891,1012.746232,1.387465,218.213758,0.0,NaN,NaN,NaN,0.091138,2013,11,15,0,10
3,2013-11-1500:15:00,2.222720,80.999738,1012.925478,1.803502,224.199997,0.0,NaN,NaN,NaN,0.097796,2013,11,15,0,15
4,2013-11-1500:20:00,2.135194,81.082619,1012.874290,1.640076,225.130637,0.0,NaN,NaN,NaN,0.096651,2013,11,15,0,20


In [46]:
data.columns

Index(['Aeg', 'Temperatuur', 'Niiskus', 'Õhurõhk', 'Tuulekiirus', 'Tuulesuund',
       'Sademed', 'UVindeks', 'Valgustatus', 'Kiirgusvoog', 'Radioaktiivsus',
       'Aasta', 'Kuu', 'Päev', 'Tund', 'Minut'],
      dtype='object')

In [48]:
data = data[["Aasta","Kuu","Päev","Tund","Minut","Temperatuur","Niiskus","Õhurõhk","Tuulekiirus","Tuulesuund","Sademed","Kiirgusvoog"]] # "Sadanudlumi"
#data['Sadanudlumi'] = data['Sadanudlumi'].fillna(0)
#data.columns = ["Aasta","Kuu","Päev","Tund","Minut","Temperatuur","Niiskus","Õhurõhk","Tuule kiirus","Tuule suund","Sademed","Kiirgusvoog"]
data['Sademed'] = data['Sademed'].fillna(0)
data['Tuulekiirus'] = data['Tuulekiirus'].fillna(method='backfill')
data['Tuulesuund'] = data['Tuulesuund'].fillna(method='backfill')
data['Temperatuur'] = data['Temperatuur'].fillna(method='backfill')
data['Niiskus'] = data['Niiskus'].fillna(method='backfill')
data['Õhurõhk'] = data['Õhurõhk'].fillna(method='backfill')
data['Kiirgusvoog'] = data['Kiirgusvoog'].fillna(method='backfill')
data["KiirgusvoogTulevikus"] = data["Kiirgusvoog"].shift(-1)
data = data.dropna()
data.isnull().sum()

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_1535/162623222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sademed'] = data['Sademed'].fillna(0)
/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_1535/162623222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tuulekiirus'] = data['Tuulekiirus'].fillna(method='backfill')
/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_1535/162623222.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

Aasta                   0
Kuu                     0
Päev                    0
Tund                    0
Minut                   0
Temperatuur             0
Niiskus                 0
Õhurõhk                 0
Tuulekiirus             0
Tuulesuund              0
Sademed                 0
Kiirgusvoog             0
KiirgusvoogTulevikus    0
dtype: int64

In [55]:
data = data.rename(columns = {
    'Aasta':'year',
    'Kuu':'month',
    'Päev':'day',
    'Tund':'hour',
    'Minut':'minute',
    'Temperatuur':'temperature',
    'Niiskus':'humidity',
    'Õhurõhk':'atmospheric_pressure',
    'Tuulekiirus':'wind_speed',
    'Tuulesuund':'wind_direction',
    'Sademed':'precipitation',
    #'Sadanudlumi':'snow',
    'Kiirgusvoog':'radiation_flux',
    'KiirgusvoogTulevikus':'rad_flux_infuture'    
    })
data.columns

Index(['year', 'month', 'day', 'hour', 'minute', 'temperature', 'humidity',
       'atmospheric_pressure', 'wind_speed', 'wind_direction', 'precipitation',
       'radiation_flux', 'rad_flux_infuture'],
      dtype='object')

In [50]:
data[data.humidity == data.humidity.max()]

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,Tuulekiirus,Tuulesuund,precipitation,radiation_flux,rad_flux_infuture
51611,2014,5,13,11,0,8.624431,117.297204,990.984611,3.325451,290.185328,0.0,0.0,0.0
51611,2014,5,13,11,0,8.624431,117.297204,990.984611,3.325451,290.185328,0.0,0.0,0.0


In [56]:
data.describe()

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
count,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000,811085.000000
mean,2017.398950,6.494621,15.751352,11.481656,27.512518,6.825025,75.458473,1010.824081,3.258511,199.267411,0.004576,89.464947,89.464947
std,2.424143,3.526065,8.820120,6.926916,17.262782,8.905742,19.869568,11.040802,1.678265,91.412932,0.050455,181.339887,181.339887
min,2013.000000,1.000000,1.000000,0.000000,0.000000,-24.307660,0.000000,963.557524,0.000000,0.000000,0.000000,-2.625525,-2.625525
25%,2016.000000,3.000000,8.000000,5.000000,14.000000,0.635374,62.394290,1003.885773,2.053779,124.554193,0.000000,0.000000,0.000000
50%,2018.000000,6.000000,16.000000,11.000000,30.000000,5.843931,81.315327,1011.116467,3.025117,224.132079,0.000000,2.639460,2.639460
75%,2019.000000,10.000000,23.000000,17.000000,45.000000,13.633509,91.499743,1018.149000,4.240506,274.416061,0.000000,74.003317,74.003317
max,2021.000000,12.000000,31.000000,23.000000,59.000000,33.553939,117.297204,1045.714767,15.497177,360.000000,9.799000,1132.409778,1132.409778


In [57]:
data.sample(50)

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
1405,2016,11,19,21,5,5.124167,90.012525,1014.736533,1.492448,167.819459,0.000,0.000000,0.000000
94037,2016,10,6,12,20,6.005477,78.302563,1029.698467,7.070330,69.678969,0.000,107.825320,123.071413
53005,2014,5,18,7,10,13.316553,55.682777,1011.567271,1.644792,65.937943,0.000,0.000000,0.000000
63479,2019,6,18,12,35,21.336877,53.585068,1016.905200,5.590781,274.324172,0.000,861.046841,834.383963
10437,2016,12,21,5,35,1.388592,96.355218,1025.617442,5.874508,272.858168,0.000,1.955190,1.975483
7904,2019,12,12,10,40,0.120362,79.565520,1011.938167,2.144881,183.181571,0.000,82.005115,84.409702
88599,2019,9,13,17,55,13.433324,93.818571,1008.847533,3.664375,278.753632,0.000,101.041133,130.957801
63102,2021,6,13,17,45,16.494863,72.841374,1008.525333,6.588624,345.945987,0.000,346.726557,183.401602
12761,2017,12,29,7,25,2.040000,88.681432,1000.083733,2.317934,194.094896,0.000,4.854070,4.959125
38949,2019,3,25,7,25,0.187146,90.296050,1004.493767,5.732482,276.789361,0.000,108.533753,111.294707


In [58]:
data.head(30)

,year,month,day,hour,minute,temperature,humidity,atmospheric_pressure,wind_speed,wind_direction,precipitation,radiation_flux,rad_flux_infuture
0,2013,11,15,0,0,2.253725,80.280623,1012.974940,1.396552,219.132229,0.0,0.0,0.0
1,2013,11,15,0,5,2.269765,80.633583,1013.042850,1.246345,220.840096,0.0,0.0,0.0
2,2013,11,15,0,10,2.259058,80.844891,1012.746232,1.387465,218.213758,0.0,0.0,0.0
3,2013,11,15,0,15,2.222720,80.999738,1012.925478,1.803502,224.199997,0.0,0.0,0.0
4,2013,11,15,0,20,2.135194,81.082619,1012.874290,1.640076,225.130637,0.0,0.0,0.0
5,2013,11,15,0,25,2.075500,81.316279,1012.867377,1.369254,221.732026,0.0,0.0,0.0
6,2013,11,15,0,30,2.041885,81.697612,1012.920600,1.314422,223.314140,0.0,0.0,0.0
7,2013,11,15,0,35,2.038886,82.163594,1013.180392,1.228393,221.865827,0.0,0.0,0.0
8,2013,11,15,0,40,2.022422,82.593353,1013.001292,1.528448,219.857440,0.0,0.0,0.0
9,2013,11,15,0,45,2.020750,83.062526,1012.941358,1.403501,223.718700,0.0,0.0,0.0


In [59]:
data.to_csv("data.csv")

## trying to get data directly from web page

In [18]:
#url = 'https://meteo.physic.ut.ee/et/archive.php?do=data&begin%5Byear%5D=2016&begin%5Bmon%5D=11&begin%5Bmday%5D=18&end%5Byear%5D=2021&end%5Bmon%5D=11&end%5Bmday%5D=18&9=1&12=1&10=1&15=1&16=1&14=1&snow_16=1&ok=+Esita+päring+'


def get_weather_data(start_year, start_month, start_day, end_year, end_month, end_day):
    url = f'https://meteo.physic.ut.ee/et/archive.php?do=data&begin%5Byear%5D={start_year}&begin%5Bmon%5D={start_month}&begin%5Bmday%5D={start_day}&end%5Byear%5D={end_year}&end%5Bmon%5D={end_month}&end%5Bmday%5D={end_day}&9=1&12=1&11=1&14=1&ok=+Esita+p%C3%A4ring+'
    c = pd.read_csv(url)
    #c.columns = ['Aeg', 'Temperatuur', 'Niiskus', 'Valgustatus', 'Kiirgusvoog']
    return c

In [19]:
train = get_weather_data(2011, 1, 1, 2015, 12, 12)

In [20]:
train.head()

,Aeg,Temperatuur,Niiskus,Valgustatus,Kiirgusvoog
0,2011-01-01 00:00:00,-6.936343679417071,98.6784171261325,1354.9408873569798,
1,2011-01-01 00:05:00,-6.9568330698942695,98.59701577484581,1029.54357837049,
2,2011-01-01 00:10:00,-6.97364936736915,98.42287783684701,1418.1783680055298,
3,2011-01-01 00:15:00,-6.94325190753387,98.49124327899729,1294.6821264335601,
4,2011-01-01 00:20:00,-6.86547316481287,98.64064185206159,1374.1625144454501,


In [17]:
train.isnull().sum()

Aeg            0
Temperatuur    0
Niiskus        0
Valgustatus    0
Kiirgusvoog    0
dtype: int64